In [1]:
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
import xarray as xr
import pickle

sys.path.append('D:\\mehdi\\WaterBalance_new\\')
from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *


In [2]:
data_path = "D:\\mehdi\\DailyData\\Data\\"

# list of name of files in folder
def list_of_files(data_path):    
    files = os.listdir(data_path)
    return files

all_list_files = list_of_files(data_path)

# PET

In [3]:
para = ["Tmin", "Tmax"]

list_files = {}

for p in para:
    
    tmp = pd.DataFrame(
        {
            "file": [x for x in all_list_files if x.startswith(p)],
            "year": [int(x.split("_")[1][0:4]) for x in all_list_files if x.startswith(p)]
        }
        
    )
    
    tmp.sort_values(by = "year", inplace = True)
    
    list_files[p] = tmp



In [8]:
selected_years = [2010, 2011]

for year in selected_years:
    
    if year == selected_years[0]:
        pass
    else:
        year_tmp = year - 1
        f"Tmin_{year_tmp}.nc}"
        pass
    
    Tmin = xr.open_dataset(f"{data_path}{list_files['Tmin'][list_files['Tmin']['year'] == year]['file'].values[0]}")
    
    Tmax = xr.open_dataset(f"{data_path}{list_files['Tmax'][list_files['Tmax']['year'] == year]['file'].values[0]}")
    
    lat = np.repeat(Tmin.lat, Tmin.lon.size).values.reshape(Tmin.lat.size, Tmin.lon.size)
    
    lon = np.repeat(Tmin.lon, Tmin.lat.size).values.reshape(Tmin.lat.size, Tmin.lon.size)
    
    Ra = Tmin.copy(deep = True)
    
    Ra = Ra.rename({'air_temperature': 'extraterrestrial_radiation'})
    
    ET = Tmin.copy(deep = True)
    
    ET = ET.rename({'air_temperature': 'et_hargreaves_samani'})
    
    for j_day in range(1, Tmin.time.size + 1):
        
        if j_day != 1:
            pass
        
        print(year, j_day)
        
        Tmin_day = Tmin.to_array()[0, j_day - 1, :, :]
        
        Tmax_day = Tmax.to_array()[0, j_day - 1, :, :]
        
        Tmean_day = (Tmin_day + Tmax_day) / 2
        
        ra_day = np.vectorize(extraterrestrial_radiation)(
            inverse_relative_distance_earth_sun(
                julian_date = j_day
            ),
            sunset_hour_angle(
                latitude=convert_degrees2radians(lat),
                solar_declination = solar_declination(
                    julian_date = j_day
                )
            ),
            convert_degrees2radians(
                degrees = lat
            ),
            solar_declination(
                julian_date = j_day
            )            
        )
        
        Ra["extraterrestrial_radiation"][j_day - 1, :, :] = ra_day
                
        et_day_hargreaves_samani = np.vectorize(ReferenceEvapotranspiration.hargreaves_samani)(
            tmin = Tmin_day,
            tmax = Tmax_day,
            tmean = Tmean_day,
            ra = ra_day
        )
        
        ET["et_hargreaves_samani"][j_day - 1, :, :] = et_day_hargreaves_samani
        
    Ra.to_netcdf(f"{data_path}Ra_{year}.nc")
    
    ET.to_netcdf(f"{data_path}ET_{year}.nc")
        
    
    break


2010 1
2010 2
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2010 11
2010 12
2010 13
2010 14
2010 15
2010 16
2010 17
2010 18
2010 19
2010 20
2010 21
2010 22
2010 23
2010 24
2010 25
2010 26
2010 27
2010 28
2010 29
2010 30
2010 31
2010 32
2010 33
2010 34
2010 35
2010 36
2010 37
2010 38
2010 39
2010 40
2010 41
2010 42
2010 43
2010 44
2010 45
2010 46
2010 47
2010 48
2010 49
2010 50
2010 51
2010 52
2010 53
2010 54
2010 55
2010 56
2010 57
2010 58
2010 59
2010 60
2010 61
2010 62
2010 63
2010 64
2010 65
2010 66
2010 67
2010 68
2010 69
2010 70
2010 71
2010 72
2010 73
2010 74
2010 75
2010 76
2010 77
2010 78
2010 79
2010 80
2010 81
2010 82
2010 83
2010 84
2010 85
2010 86
2010 87
2010 88
2010 89
2010 90
2010 91
2010 92
2010 93
2010 94
2010 95
2010 96
2010 97
2010 98
2010 99
2010 100
2010 101
2010 102
2010 103
2010 104
2010 105
2010 106
2010 107
2010 108
2010 109
2010 110
2010 111
2010 112
2010 113
2010 114
2010 115
2010 116
2010 117
2010 118
2010 119
2010 120
2010 121
2010 122
2010 123
2

In [22]:
ET.to_netcdf(f"{data_path}ET_{year}.nc")

In [ ]:
Ra = Ra.rename({'air_temperature': 'temp'})

In [ ]:
Ra

In [ ]:
xr.DataArray(
            Tmin.air_temperature,
            coords={
                "time" : Tmin.time.values,
                "lat" : Tmin_day.lat.values,
                "lon" : Tmin_day.lon.values,
            }, 
            dims=['time', 'lat', 'lon']
        )

In [ ]:
Tmin

In [ ]:
with open('D:\\mehdi\\DailyData\\Tmax\\Tmax-30.pkl', 'rb') as f:
    t_max11 = pickle.load(f)

with open('D:\\mehdi\\DailyData\\Tmax\\Tmax-31.pkl', 'rb') as f:
    t_max12 = pickle.load(f)


concatenated_data = xr.concat([t_max11, t_max12], dim='time')
sliced_dataset = concatenated_data.sel(time=slice("2010-01-01", "2010-12-31"))



reshaped_data = result_ET_array.reshape(365,200, 600)

ds_array = sliced_dataset.assign(ET=(['time', 'lat', 'lon'], reshaped_data))

ds_array

In [ ]:
Tmin_day

In [ ]:
# result_array.size
# ET_ca_array = result_array.reshape(365,200,600)
t = Tmin.assign(ra=(['time', 'lat', 'lon'], result_array))
t

In [ ]:
yy = xr.open_dataset("D:\\mehdi\\DailyData\\Data\\Tmax_2010.nc")
yy.sel(time = "2010-01-01")

In [ ]:
def check_between(
    a: float,
    min: float,
    max: float,
    name: str,
):
    if a < min or a > max:
        raise ValueError(f"{name} must be between {min} and {max}!")

In [ ]:
Tmean_day

In [ ]:
for year in list_of_year[:2]:
    t_min_{year} = xr.open_dataset(f"Data\\Tmin_{1979}.nc")
    t_max_{year} = xr.open_dataset(f"Data\\Tmax_{1979}.nc")
    
    for i in range(t_min.sizes["time"]):
        print(i)

In [ ]:
import os
import sys

sys.path.append('D:\\mehdi\\WaterBalance_new\\')

from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *

In [ ]:
time_list = a['time'].values.astype(str).tolist()

In [ ]:
array_t_min = a.air_temperature.to_numpy()
array_t_min

In [ ]:
vectorized_func = np.vectorize(standard_date_to_Julian_day)
result = vectorized_func(array_t_min)
result